<a href="https://colab.research.google.com/github/tommasocarzaniga/CNM_CycNucMed/blob/main/EMBA_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cyclotrons for Nuclear Medicine

###Setup
First, make the necessary imports.
Note that further imports may have to be made in addition to the ones below, if your application uses additional fetures such as loaders and tools. You can find the code for these imports in the respective sections of the tutorial notebooks.

In [6]:
!pip install -q langchain langchain-community langchain-core langchain-openai langchain-huggingface

from google.colab import userdata
from langchain.chains import SequentialChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_openai import ChatOpenAI
import os
import pprint
import getpass
from IPython.display import Markdown
import IPython.display as ipd
from PIL import Image
import urllib.request

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 37.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.1 requires langchain-core<2.0.0,>=1.2.5, but you have langchain-core 0.2.43 which is incompatible.
langchain-classic 1.0.1 requires langchain-text-splitters<2.0.0,>=1.1.0, but you have langchain-text-splitters 0.2.4 which is incompatible.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.2.43 which is incompatible.


Then, assign the API keys to be able to use OpenAI, Google Serper, Huggingface, etc.

When working with sensitive information like API keys or passwords in Google Colab, it's crucial to handle data securely. As you learnt in the tutorial session, two common approaches for this are using **Colab's Secrets Manager**, which stores and retrieves secrets without exposing them in the notebook, and `getpass`, a Python function that securely prompts users to input secrets during runtime without showing them. Both methods help ensure your sensitive data remains protected.

In [3]:
#You can remove the keys you will not use

#When using Colab Secret Manager
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
#When using getpass
#os.environ['OPENAI_API_KEY'] = getpass.getpass()

#When using Colab Secret Manager
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
#When using getpass
#os.environ['SERPER_API_KEY'] = getpass.getpass()

#When using Colab Secret Manager
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
#When using getpass
#os.environ['HF_TOKEN'] = getpass.getpass()

NameError: name 'userdata' is not defined

In [ ]:
!apt-get update
!apt-get install -y \
  libatk1.0-0 \
  libatk-bridge2.0-0 \
  libcups2 \
  libdrm2 \
  libxkbcommon0 \
  libxcomposite1 \
  libxdamage1 \
  libxfixes3 \
  libxrandr2 \
  libgbm1 \
  libpango-1.0-0 \
  libcairo2 \
  libasound2

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 2s (1,749 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2 is already the newest version (1.2.6.1-

In [ ]:
!pip install playwright pandas
!playwright install chromium

In [ ]:
from playwright.async_api import async_playwright

async def test_playwright():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto("https://nucleus.iaea.org/sites/accelerators/Pages/Cyclotron.aspx")
        print(await page.title())
        await browser.close()

await test_playwright()


Pages - Cyclotrons used for Radionuclide Production


In [ ]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright
import os
import hashlib

BASE_URL = "https://nucleus.iaea.org/sites/accelerators/Pages/Cyclotron.aspx"
HASH_PREFIX = "#InplviewHashd5afe566-18ad-4ac0-8aeb-ccf833dbc282="
PAGE_SIZE = 30

OUTPUT_CSV = os.path.join(os.getcwd(), "iaea_cyclotrons_normalized.csv")

def row_fingerprint(cells):
    return hashlib.md5(" | ".join(cells).encode("utf-8")).hexdigest()

async def scrape_all_pages():
    all_rows = []
    seen_rows = set()

    previous_new_rows_count = None

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        first_row = 1
        page_index = 0

        while True:
            if page_index == 0:
                url = BASE_URL + HASH_PREFIX
            else:
                url = (
                    BASE_URL
                    + HASH_PREFIX
                    + f"Paged%3DTRUE-p_ID%3D{first_row + PAGE_SIZE - 1}"
                    + f"-PageFirstRow%3D{first_row}"
                )

            print(f"\nLoading page {page_index + 1} (first row = {first_row})")
            await page.goto(url, timeout=60000)

            # Allow SharePoint JS to refresh the list
            await page.wait_for_timeout(3000)

            try:
                await page.wait_for_selector("table tbody tr", timeout=20000)
            except:
                print("No table found → stopping")
                break

            rows = page.locator("table tbody tr")
            n_rows = await rows.count()
            print(f"  Rows visible: {n_rows}")

            new_rows_this_page = 0

            for i in range(n_rows):
                cells = await rows.nth(i).locator("td").all_inner_texts()

                if len(cells) < 5:
                    continue

                fp = row_fingerprint(cells)

                if fp not in seen_rows:
                    seen_rows.add(fp)
                    new_rows_this_page += 1

                    all_rows.append({
                        "Country": cells[0].strip(),
                        "City": cells[1].strip(),
                        "Facility": cells[2].strip(),
                        "Manufacturer": cells[3].strip(),
                        "Model": cells[4].strip(),
                        "raw_row": " | ".join(cells)
                    })

            print(f"  New rows added: {new_rows_this_page}")

            # 🛑 YOUR REQUESTED STOP CONDITION
            if previous_new_rows_count is not None and new_rows_this_page != previous_new_rows_count:
                print(
                    f"Row count changed "
                    f"({previous_new_rows_count} → {new_rows_this_page}) → stopping"
                )
                break

            previous_new_rows_count = new_rows_this_page

            page_index += 1
            first_row += PAGE_SIZE

        await browser.close()

    df = pd.DataFrame(all_rows)
    df.to_csv(OUTPUT_CSV, index=False)

    print(f"\n✅ DONE")
    print(f"Saved {len(df)} cyclotrons to:")
    print(OUTPUT_CSV)

# Run in Jupyter
await scrape_all_pages()


Loading page 1 (first row = 1)
  Rows visible: 36
  New rows added: 31

Loading page 2 (first row = 31)
  Rows visible: 36
  New rows added: 31

Loading page 3 (first row = 61)
  Rows visible: 36
  New rows added: 26
Row count changed (31 → 26) → stopping

✅ DONE
Saved 88 cyclotrons to:
/content/iaea_cyclotrons_normalized.csv


until here is fine

In [ ]:
import pandas as pd

INPUT_CSV = "iaea_cyclotrons_table.csv"
OUTPUT_CSV = "iaea_cyclotrons_normalized.csv"

# -------------------------------------------------
# Load raw extracted table
# -------------------------------------------------
df_raw = pd.read_csv(INPUT_CSV)

# -------------------------------------------------
# Inspect available columns (optional, once)
# print(df_raw.columns)
# -------------------------------------------------

# -------------------------------------------------
# Normalization logic
# -------------------------------------------------
df_norm = pd.DataFrame()

df_norm["country"] = df_raw.get("col_1")
df_norm["city"] = df_raw.get("col_2")
df_norm["facility"] = df_raw.get("col_3")
df_norm["cyclotron_description"] = df_raw.get("col_4")

# Optional extra column if present
if "col_5" in df_raw.columns:
    df_norm["technical_notes"] = df_raw["col_5"]
else:
    df_norm["technical_notes"] = None

# Keep full raw row for traceability (important for audits)
df_norm["raw_row"] = df_raw["raw_joined"]

# -------------------------------------------------
# Basic cleanup
# -------------------------------------------------
for c in ["country", "city", "facility", "cyclotron_description"]:
    df_norm[c] = df_norm[c].astype(str).str.strip()
    df_norm[c] = df_norm[c].replace({"nan": None})

# Drop completely empty rows
df_norm = df_norm.dropna(
    how="all",
    subset=["country", "city", "facility", "cyclotron_description"]
)

# -------------------------------------------------
# Save normalized CSV
# -------------------------------------------------
df_norm.to_csv(OUTPUT_CSV, index=False)

print(f"✅ Normalized CSV saved as: {OUTPUT_CSV}")
print(f"Rows: {len(df_norm)}")

In [ ]:
# -----------------------------
# Setup browser
# -----------------------------
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

url = "https://nucleus.iaea.org/sites/accelerators/Pages/Cyclotron.aspx"
driver.get(url)

wait = WebDriverWait(driver, 30)

# -----------------------------
# Wait for table to load
# -----------------------------
wait.until(
    EC.presence_of_element_located((By.TAG_NAME, "table"))
)

time.sleep(5)  # extra safety

# -----------------------------
# Locate table rows
# -----------------------------
rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

print(f"Found {len(rows)} cyclotron entries")

data = []

# -----------------------------
# Iterate through rows
# -----------------------------
for idx in range(len(rows)):
    try:
        # Re-fetch rows each loop (DOM refresh safety)
        rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
        row = rows[idx]

        cells = row.find_elements(By.TAG_NAME, "td")
        basic_info = [c.text for c in cells]

        # Click row to open detail view
        row.click()

        # Wait for details panel/dialog
        wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='dialog'], .ms-Panel"))
        )
        time.sleep(2)

        # Extract key-value fields
        detail_blocks = driver.find_elements(By.CSS_SELECTOR, "div.ms-DetailsRow, div.ms-Panel")

        details = {}
        labels = driver.find_elements(By.CSS_SELECTOR, "span.ms-Label")
        values = driver.find_elements(By.CSS_SELECTOR, "span.ms-TextField-field")

        for l, v in zip(labels, values):
            details[l.text.strip()] = v.text.strip()

        record = {
            "Row_Index": idx,
            "Basic_Info": " | ".join(basic_info),
            **details
        }

        data.append(record)

        # Close details panel (ESC works reliably)
        driver.switch_to.active_element.send_keys("\uE00C")  # ESC key
        time.sleep(1)

    except Exception as e:
        print(f"Error on row {idx}: {e}")

# -----------------------------
# Save to CSV
# -----------------------------
df = pd.DataFrame(data)
df.to_csv("iaea_cyclotron_database.csv", index=False)

driver.quit()

print("CSV exported: iaea_cyclotron_database.csv")


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,864 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,586 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,637 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,969 kB]
Get:14 http://securit

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/chromium-browser is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x599927e2c4e3 <unknown>
#1 0x599927b5bc76 <unknown>
#2 0x599927b84d78 <unknown>
#3 0x599927b81029 <unknown>
#4 0x599927bbfccc <unknown>
#5 0x599927bbf47f <unknown>
#6 0x599927bb6de3 <unknown>
#7 0x599927b8c2dd <unknown>
#8 0x599927b8d34e <unknown>
#9 0x599927dec3e4 <unknown>
#10 0x599927df03d7 <unknown>
#11 0x599927dfab20 <unknown>
#12 0x599927df1023 <unknown>
#13 0x599927dbf1aa <unknown>
#14 0x599927e156b8 <unknown>
#15 0x599927e15847 <unknown>
#16 0x599927e25243 <unknown>
#17 0x793e1e05cac3 <unknown>


In [ ]:
import requests
import pandas as pd

TLS handshake mismatch between:

🔒 IAEA server (older / strict TLS stack, SNI quirks)

🤖 Colab Python 3.12 + OpenSSL 3.x (very strict, modern)

Install compatible version

In [ ]:
!pip install -U requests urllib3==1.26.18

Custom TLS adapter

Session with forced TLS

Fetch metadata

Build field table

Export

In [ ]:
FIELDS_URL = (
    "https://nucleus.iaea.org/sites/accelerators/"
    "_api/web/lists(guid'fcadb915-2870-44e8-b84d-b6ff9806767b')/fields"
)

params = {
    "$format": "json"
}

response = requests.get(FIELDS_URL, params=params)
response.raise_for_status()

data = response.json()["d"]["results"]

# Extract useful attributes
rows = []
for f in data:
    rows.append({
        "Title (Display Name)": f.get("Title"),
        "InternalName": f.get("InternalName"),
        "Type": f.get("TypeAsString"),
        "Hidden": f.get("Hidden"),
        "ReadOnly": f.get("ReadOnlyField")
    })

df_fields = pd.DataFrame(rows)
df_fields.shape


SSLError: HTTPSConnectionPool(host='nucleus.iaea.org', port=443): Max retries exceeded with url: /sites/accelerators/_api/web/lists(guid'fcadb915-2870-44e8-b84d-b6ff9806767b')/fields?%24format=json (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1010)')))

Create the dataset builder

In [ ]:
import pandas as pd
import re

raw_text = """
PASTE ALL PAGES 5–13 HERE EXACTLY AS YOU SENT THEM
"""

lines = [l.strip() for l in raw_text.splitlines() if l.strip()]

records = []
i = 0

while i < len(lines) - 8:
    # Heuristic: country names are always valid ISO-ish strings
    if re.match(r"^[A-Z][a-zA-Z .()'-]+$", lines[i+1]):
        record = {
            "Facility": lines[i],
            "Country": lines[i+1],
            "City": lines[i+2],
            "Manufacturer": lines[i+3],
            "Model": lines[i+4],
            "Proton energy (MeV)": lines[i+5],
            "Facility Website": lines[i+6] if "http" in lines[i+6] else "",
            "PET radionucl.": lines[i+7] if lines[i+7] in ["Yes", "No"] else "",
            "SPECT radionucl.": lines[i+8] if lines[i+8] in ["Yes", "No"] else ""
        }
        records.append(record)
        i += 9
    else:
        i += 1

df = pd.DataFrame(records)

print(f"Parsed records: {len(df)}")
df.head()

Parsed records: 0


""


Export CSV + Excel

In [ ]:
df.to_csv("public_cyclotron_db_2020_manual.csv", index=False)
df.to_excel("public_cyclotron_db_2020_manual.xlsx", index=False)

print("Files written")

Files written


Download

In [ ]:
files.download("public_cyclotron_db_2020_manual.csv")
files.download("public_cyclotron_db_2020_manual.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%writefile scrape_cyclotron_db.py
import re
import time
import pandas as pd
from playwright.sync_api import sync_playwright

LIST_URL = "https://nucleus.iaea.org/sites/accelerators/Lists/Cyclotron%20Master%20List/public_cyclotron_db_view.aspx"

COLUMNS = [
    "Facility",
    "Country",
    "City",
    "Manufacturer",
    "Model",
    "Proton energy (MeV)",
    "Facility Website",
    "PET radionucl.",
    "SPECT radionucl."
]

def clean_cell(text):
    if text is None:
        return ""
    return re.sub(r"\s+", " ", text.replace("Open Menu", "")).strip()

def extract_rows(page):
    rows = []
    # try classic table first
    tables = page.locator("table")
    for i in range(tables.count()):
        table = tables.nth(i)
        try:
            header = clean_cell(table.locator("tr").first.inner_text(timeout=500))
        except Exception:
            continue
        if "Facility" in header and "Country" in header:
            trs = table.locator("tr")
            for r in range(1, trs.count()):
                tds = trs.nth(r).locator("td")
                if tds.count() < 9:
                    continue
                cells = [clean_cell(tds.nth(c).inner_text()) for c in range(tds.count())]
                cells = [c for c in cells if c]
                if len(cells) >= 9:
                    rows.append(dict(zip(COLUMNS, cells[-9:])))
            if rows:
                return rows

    # fallback: modern grid
    grid_rows = page.locator("[role='row']")
    for r in range(grid_rows.count()):
        row = grid_rows.nth(r)
        text = clean_cell(row.inner_text())
        if not text or "Facility Country" in text:
            continue
        cells = row.locator("[role='gridcell']")
        if cells.count() >= 9:
            vals = [clean_cell(cells.nth(i).inner_text()) for i in range(cells.count())]
            vals = [v for v in vals if v]
            if len(vals) >= 9:
                rows.append(dict(zip(COLUMNS, vals[-9:])))
    return rows

def click_next(page):
    for sel in ["text=Next", "a:has-text('Next')", "button:has-text('Next')"]:
        loc = page.locator(sel)
        if loc.count():
            try:
                loc.first.click()
                return True
            except Exception:
                pass
    return False

def main():
    all_rows = []
    seen = set()

    with sync_playwright() as p:
        browser = p.chromium.launch(headless=False, slow_mo=300)
        page = browser.new_page()
        page.goto(LIST_URL, wait_until="domcontentloaded")
        page.wait_for_timeout(3000)

        try:
            acc = page.locator("text=Turn on more accessible mode")
            if acc.count():
                acc.first.click()
                page.wait_for_timeout(2000)
        except Exception:
            pass

        while True:
            page.wait_for_timeout(1500)
            rows = extract_rows(page)
            sig = str(rows[:1]) if rows else "EMPTY"
            if sig in seen:
                break
            seen.add(sig)
            all_rows.extend(rows)
            if not click_next(page):
                break

        browser.close()

    df = pd.DataFrame(all_rows).drop_duplicates()
    df.to_csv("public_cyclotron_db_2020_full.csv", index=False)
    df.to_excel("public_cyclotron_db_2020_full.xlsx", index=False)
    print("Saved files:", len(df))

if __name__ == "__main__":
    main()

Overwriting scrape_cyclotron_db.py


In [ ]:
!python scrape_cyclotron_db.py

Traceback (most recent call last):
  File "/content/scrape_cyclotron_db.py", line 111, in <module>
    main()
  File "/content/scrape_cyclotron_db.py", line 79, in main
    browser = p.chromium.launch(headless=False, slow_mo=300)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/playwright/sync_api/_generated.py", line 14568, in launch
    self._sync(
  File "/usr/local/lib/python3.12/dist-packages/playwright/_impl/_sync_base.py", line 115, in _sync
    return task.result()
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/playwright/_impl/_browser_type.py", line 98, in launch
    await self._channel.send(
  File "/usr/local/lib/python3.12/dist-packages/playwright/_impl/_connection.py", line 69, in send
    return await self._connection.wrap_api_call(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/playwright/_impl/_connection.py", line 559, in wrap_api_cal

In [ ]:
import pandas as pd

df_test = pd.DataFrame({
    "Facility": ["TEST"],
    "Country": ["TEST"],
    "City": ["TEST"]
})

df_test.to_csv("test_download.csv", index=False)
df_test.to_excel("test_download.xlsx", index=False)

print("Files written")

Files written


In [ ]:
from google.colab import files
files.download("test_download.csv")
files.download("test_download.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("public_cyclotron_db_2020_full.csv")
files.download("public_cyclotron_db_2020_full.xlsx")

FileNotFoundError: Cannot find file: public_cyclotron_db_2020_full.csv